In [ ]:
#Data____
#Authors: Rohan, Dakota, Chris, David
#Organize/clean the raw annotated data
import numpy as np
import pandas as pd;
import matplotlib.pyplot as plt
import tensorflow as tf

Thought process on working with the data:

We have the true predicted in the annotated data set as G, the annotators broke up the by selected text in the json.
So we decided to map selected text by policy ID further by that particular segmented ID with the class of G (10 class categorical)


In [ ]:
#cleaning and organzing the data
filenames = ["/content/Annotations/1017_sci-news.com.csv", "/content/Annotations/1028_redorbit.com.csv","/content/Annotations/1050_honda.com.csv","/content/Annotations/105_amazon.com.csv","/content/Annotations/1106_allstate.com.csv",
             "/content/Annotations/1468_rockstargames.com.csv", "/content/Annotations/175_mlb.mlb.com.csv", "/content/Annotations/303_reddit.com.csv", "/content/Annotations/32_voxmedia.com.csv", "/content/Annotations/635_playstation.com.csv","/content/Annotations/856_sciencemag.org.csv",
             "/content/Annotations/988_solarviews.com.csv" ]

In [ ]:
#create the dataFrames
totalData = 0
headers = ['AnnotationID', 'BATCHID', 'ANNOTATORID', 'POLICYID', 'SEGMENTID', 'categoryName','attributeValuePairs','PolicyURL','date']

# Assuming 'filenames' is a list of filenames containing your data
for i, filename in enumerate(filenames, 1):
    globals()[f'data_{i}'] = pd.read_csv(filename)
    globals()[f'data_{i}'] = pd.DataFrame(globals()[f'data_{i}'])
    globals()[f'data_{i}'] = globals()[f'data_{i}'].rename(columns=dict(zip(globals()[f'data_{i}'].columns, headers)))
    totalData += 1

#print(globals()[f'data_{1}'].head())
i = 1

while i <= totalData:
    print(f'The head of the {i} ')
    if isinstance(globals()[f'data_{i}'], pd.DataFrame):
        df = globals()[f'data_{i}']
        # Dropping specified columns
        columns_to_drop = ["AnnotationID", "BATCHID", "ANNOTATORID", "date"]
        df = df.drop(columns=columns_to_drop, axis=1)

        globals()[f'data_{i}'] = df
        #print(globals()[f'data_{i}'].head())

    i += 1





The head of the 1 
The head of the 2 
The head of the 3 
The head of the 4 
The head of the 5 
The head of the 6 
The head of the 7 
The head of the 8 
The head of the 9 
The head of the 10 
The head of the 11 
The head of the 12 


So Now That we have data frames for 12 urls, we want to parse the json column to extract the line that generated the categoryName.

In [ ]:
import json

i = 1
while i <= totalData:
    selected_texts = []
    df = globals()[f'data_{i}']

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        attribute_pairs = json.loads(row['attributeValuePairs'])

        # Iterate over the actual json array
        for key, value in attribute_pairs.items():
            selected_text = value.get('selectedText', '')
            selected_texts.append(selected_text)
        df["ActualText"] = selected_text
        globals()[f'data{i}'] = df
    #Add the extracted selected texts to a new column named "ActualText"
    #df['ActualText'] = selected_texts

    # Update the dataframe in globals()
    globals()[f'data{i}'] = df

    # Print the "ActualText" column for debugging
    print(globals()[f'data{i}'].head())

    i += 1  # Increment i to move to the next dataframe






   POLICYID  SEGMENTID                categoryName  \
0      3905          1  First Party Collection/Use   
1      3905          1  First Party Collection/Use   
2      3905          2              Data Retention   
3      3905          3  First Party Collection/Use   
4      3905          3  First Party Collection/Use   

                                 attributeValuePairs      PolicyURL  \
0  {"Collection Mode": {"selectedText": "nformati...  Not specified   
1  {"Collection Mode": {"selectedText": "nformati...  Not specified   
2  {"Personal Information Type": {"selectedText":...  Not specified   
3  {"Collection Mode": {"selectedText": "Not sele...  Not specified   
4  {"Collection Mode": {"selectedText": "Not sele...  Not specified   

              ActualText  
0  privacy@sci-news.com.  
1  privacy@sci-news.com.  
2  privacy@sci-news.com.  
3  privacy@sci-news.com.  
4  privacy@sci-news.com.  
   POLICYID  SEGMENTID                categoryName  \
0      3906          1          

In [ ]:
#standardize the text data
for i in range(1, totalData + 1):
    if f'data_{i}' in globals() and isinstance(globals()[f'data_{i}'], pd.DataFrame):
        df = globals()[f'data_{i}']

        #remove grammar and convert to lowercase
        df['ActualText'] = df['ActualText'].apply(lambda x: x.lower().replace(',', '').replace('.', '').replace('!', '').replace('?', ''))
        globals()[f'data_{i}'] = df
    #print(globals()[f'data_{i}'].head())


   POLICYID  SEGMENTID                categoryName  \
0      3905          1  First Party Collection/Use   
1      3905          1  First Party Collection/Use   
2      3905          2              Data Retention   
3      3905          3  First Party Collection/Use   
4      3905          3  First Party Collection/Use   

                                 attributeValuePairs      PolicyURL  \
0  {"Collection Mode": {"selectedText": "nformati...  Not specified   
1  {"Collection Mode": {"selectedText": "nformati...  Not specified   
2  {"Personal Information Type": {"selectedText":...  Not specified   
3  {"Collection Mode": {"selectedText": "Not sele...  Not specified   
4  {"Collection Mode": {"selectedText": "Not sele...  Not specified   

            ActualText  
0  privacy@sci-newscom  
1  privacy@sci-newscom  
2  privacy@sci-newscom  
3  privacy@sci-newscom  
4  privacy@sci-newscom  
   POLICYID  SEGMENTID                categoryName  \
0      3906          1                      

# MODEL

In [ ]:
#!pip install transformers

## Get all the imports

In [ ]:
import transformers
import torch
from transformers import AdamW
from transformers import BertTokenizer,\
BertForSequenceClassification

### Import Pretrained model

In [ ]:
#import pretrained model

pretrained_model_name = 'bert-base-uncased'

### Load pre-learned model

In [ ]:
#Loading the corpus of pre-learned. By loading this pre-trained model, we are leveraging the knowledge and insights gained from its pre-training.
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification.from_pretrained(pretrained_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [ ]:
input_texts = df['ActualText']
labels = df['categoryName']

input_ids = []
attention_masks = []

for text in input_texts:
	encoded_dict = tokenizer.encode_plus(
		text,
		add_special_tokens=True,
		max_length=128,
		padding='max_length',
		truncation=True,
		return_tensors='pt'
	)

	input_ids.append(encoded_dict['input_ids'])
	attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)